We will do some preliminary data processing on the congestion scoot data. This data is downloaded from https://roads.data.tfl.gov.uk/. This data contains all the traffic congestion data from December 17, 2017 to March 16, 2018. We will do some preliminary processing on this data.


我们对 congestion scoot 这个数据进行初步的预处理。这个数据下载来自 https://roads.data.tfl.gov.uk/ 。这个数据包含了从2017年12月17日到2018年3月16日所有的交通拥堵数据。我们将这个数据进行初步的处理。

In [1]:
# import libraries

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
import contextily as ctx

import os
import glob

First, we will process the metadata. The metadata is a csv file that contains the observation point id and their coordinate information (represented in the British National Grid coordinate system).

首先是对metadata的处理，metadata是一个csv文件，里面包含了所有记录道路拥堵数据的观测点id以及他们的坐标信息（基于British National Grid坐标系统进行表示）。

In [2]:

file_path = '../Data/CongestionScoot/Metadata.csv'

# read the data
# 读取 CSV 文件
df = pd.read_csv(file_path)

df.sample(5)

,ID,Easting,Northing
1168,06-034,544674,178944
3928,30-108,522736,187403
1838,09-411,529257,175214
1665,09-059,530186,170859
980,05-065,533956,180953


In [3]:
# make sure the Easting and Northing columns are in the correct data type
# 确保 Easting 和 Northing 列的数据类型是正确的
df['Easting'] = pd.to_numeric(df['Easting'], errors='coerce')
df['Northing'] = pd.to_numeric(df['Northing'], errors='coerce')

In [9]:

# 创建点几何列
geometry = [Point(xy) for xy in zip(df['Easting'], df['Northing'])]

# 创建 GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Easting'], df['Northing']))


# 设置坐标参考系统(CRS)为英国国家网格参考系统 (EPSG:27700)，并转换为WGS84 (EPSG:4326)
gdf.crs = 'epsg:27700'
gdf = gdf.to_crs(epsg=4326)

# 创建 folium 地图对象
m = folium.Map(
    location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()], # 设置地图的中心点
    zoom_start=10 # 初始缩放级别
)


# 在地图上添加点
for _, row in gdf.iterrows():
    
    # 创建自定义图标
    icon = folium.CustomIcon(
    "location-crosshairs-solid.svg",
    icon_size=(20, 20),
    icon_anchor=(10, 10)
)
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        icon = icon,
        popup=row['ID'] # 假设每个点有一个 'id' 列作为标识
    ).add_to(m)

# 添加图层切换控件
folium.LayerControl().add_to(m)

# 保存地图到HTML文件
m.save('map.html')

In [ ]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

# 准备数据
data = pd.read_csv('../Data/CongestionScoot/Metadata.csv')  # 假设您的数据在'data.csv'文件中

# 创建地图对象
m = folium.Map(
    location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()], # 设置地图的中心点
    zoom_start=10 # 初始缩放级别
)

# 添加图层
for index, row in data.iterrows():
    folium.Marker([row['lat'], row['lon']], popup=row['name']).add_to(m)

# 添加图层切换控件
folium.LayerControl().add_to(m)

# 保存地图到HTML文件
m.save('map.html')

首先需要对三个文件夹中的同名文件进行合并，然后对合并后的文件进行处理。

In [43]:
# 文件夹路径
folders = ['../Data/CongestionScoot/CSV-171217-180115', '../Data/CongestionScoot/CSV2-180116-180214', '../Data/CongestionScoot/CSV3-180215-180316']
new_folder = '../Data/CongestionScoot/MergedCSVs'

# 确保输出文件夹存在
if not os.path.exists(new_folder):
    os.makedirs(new_folder)


In [44]:

# 获取每个文件夹中的文件列表
files_in_folders = [set(glob.glob(f"{folder}/*.csv")) for folder in folders]

# 找出所有文件夹中共有的文件名
common_files = set.intersection(*files_in_folders)



In [49]:
# 定义所有观测点的ID列表
all_ids = df['ID'].unique()

In [50]:

# 对于每个共有的文件
for id in all_ids:
    
    # 读取并合并文件
    dfs = []
    for folder in folders:
        file_path = f"{folder}/{id}.csv"
        # 忽略每个文件的前4行
        df = pd.read_csv(file_path, skiprows=4)
        dfs.append(df)
    merged_df = pd.concat(dfs, ignore_index=True)
    
    # 保存到新文件夹
    merged_df.to_csv(f"{new_folder}/{id}.csv", index=False)

print("Merging completed.")

FileNotFoundError: [Errno 2] No such file or directory: '../Data/CongestionScoot/CSV-171217-180115/01-857.csv'

Since every obervation's recording is in a separate csv file, we will convert the metadata from long table to wide table. This will make it easier for us to process the time series data of each observation point.

因为每个记录点的congestion信息都是按照时间进行记录的，而每个记录点是一个单独的csv文件。如果需要对某个观测点的时间序列数据进行处理，我们需要对原csv文件进行某些预处理，将long table转换为wide table。这样我们就可以对某个观测点的时间序列数据进行处理。


In [25]:
# 因为csv文件前几行是无效的，所以需要跳过前几行
ObservationPoint = pd.read_csv('../Data/CongestionScoot/CSV-171217-180115/00-005.csv', skiprows=4)
ObservationPoint.sample(8)

,DateTime,Date,Time,SatMean,SatBand,FlowMean
2006,43106.885417,6-Jan-2018,21:15,63.75,0-79%,3899
1291,43099.437500,30-Dec-2017,10:30,52.25,0-79%,2826
187,43087.947917,18-Dec-2017,22:45,64.25,0-79%,3883
2278,43109.718750,9-Jan-2018,17:15,85.00,80-89%,4487
2733,43114.458333,14-Jan-2018,11:00,86.25,80-89%,4978
457,43090.750000,21-Dec-2017,18:00,103.25,>= 100%,5529
2544,43112.489583,12-Jan-2018,11:45,51.00,0-79%,2727
237,43088.468750,19-Dec-2017,11:15,90.75,90-99%,5973


In [39]:
ObservationPoint_pivot = ObservationPoint.pivot_table(index='Date', columns='Time', values=['FlowMean','SatMean'])
ObservationPoint_pivot.sample(9)

FlowMean                                                        \
Time           00:00 00:15 00:30 00:45 01:00 01:15 01:30 01:45 02:00 02:15   
Date                                                                         
5-Jan-2018      1968  2829  2070  1865  1773  1684  1538  1466  1537  1379   
17-Dec-2017     4835  5223  5189  3827  5176  4365  4323  3789  3655  4081   
13-Jan-2018     4133  4231  3932  4025  3678  3344  3166  3069  3275  2870   
20-Dec-2017     4263  3484  3547  3371  2813  2881  2328  2630  1969  2014   
26-Dec-2017     2526  2602  2149  1785  1432  1352  1318   986  1109   861   
27-Dec-2017     2201  1763  2103  1857  1597  1683  1514  1354  1168  1217   
2-Jan-2018      1816  1449  1613  1475  1364  1203  1039  1172   983   805   
25-Dec-2017     2931  3133  2645  2614  2448  2366  1972  1893  1374  1481   
23-Dec-2017     3597  3493  3005  2749  3035  2773  2547  2642  2403  2629   

             ... SatMean                                                   \
Time         ...   21:30  21:45  22:00  22:15  22:30  22:45  23:00  23:15   
Date         ...                                                            
5-Jan-2018   ...   63.50  47.50  61.50  74.00  56.75  65.50  57.25  54.75   
17-Dec-2017  ...   56.50  53.75  46.75  54.50  51.75  46.50  55.50  53.50   
13-Jan-2018  ...   75.25  64.50  70.00  73.25  79.75  76.75  69.00  62.00   
20-Dec-2017  ...   72.00  65.50  72.00  70.75  79.25  73.25  70.50  72.00   
26-Dec-2017  ...   63.00  47.75  55.50  63.00  67.75  53.25  54.25  42.00   
27-Dec-2017  ...   44.75  45.25  46.50  50.50  52.25  48.25  40.50  44.25   
2-Jan-2018   ...   44.00  45.25  45.75  55.25  43.75  34.50  32.75  30.00   
25-Dec-2017  ...   72.25  66.00  60.25  64.75  66.25  64.75  54.50  46.00   
23-Dec-2017  ...   61.50  58.50  53.50  57.50  69.50  63.75  57.00  55.50   

                           
Time         23:30  23:45  
Date                       
5-Jan-2018   54.75  55.50  
17-Dec-2017  54.00  48.50  
13-Jan-2018  67.50  71.00  
20-Dec-2017  58.00  60.00  
26-Dec-2017  32.25  33.50  
27-Dec-2017  40.50  32.75  
2-Jan-2018   32.00  39.75  
25-Dec-2017  52.50  37.75  
23-Dec-2017  59.75  58.75  

[9 rows x 192 columns]